[Multi Label BERT Fine-Tuning](https://curiousily.com/posts/multi-label-text-classification-with-bert-and-pytorch-lightning/)

[Multi Label distilBERT Fine-Tuning](https://medium.com/analytics-vidhya/finetune-distilbert-for-multi-label-text-classsification-task-994eb448f94c)

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install -U transformers datasets lightning clean-text wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB

In [3]:
# parameters

epochs = 10
learning_rate = 1e-5
max_length = 500
batch_size = 16
num_workers = 15
train_sample = 'balanced toxic data set'
model_name = 'distilbert-base-uncased'
task_folder= "Multi Label Hate Speech Bullying NSFW"
LABEL_COLUMNS = ["nsfw", "hate_speech", "bullying"]
n_labels = len(LABEL_COLUMNS)
wandb_name = f"kaggle - {model_name} - only_one_top_layer - train sample {train_sample} - epochs {epochs} - batch size {batch_size} - max_length {max_length} learning rate {str(learning_rate)} num_workers {num_workers}_with_all_accuracy"
wandb_project = task_folder
pickle_name = f"kaggle - {model_name}_only_one_top_layer_train_sample_{train_sample}_epochs_{epochs}_batch_size_{batch_size}_max_length_{max_length}_learning_rate_{str(learning_rate)}_num_workers_{num_workers}_with_all_accuracy"

In [4]:
import wandb

wandb.init(project=wandb_project, name=wandb_name, group="yoav_yosef")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
import pandas as pd

df_train = pd.read_csv("/content/drive/MyDrive/BIU - Final Project/Data/kaggle_train_toxic_final_balanced.csv")
df_val = pd.read_csv("/content/drive/MyDrive/BIU - Final Project/Data/kaggle_val_toxic_final_balanced.csv")
df_test = pd.read_csv("/content/drive/MyDrive/BIU - Final Project/Data/kaggle_test_toxic_final_balanced.csv")

In [6]:
sum(df_train[LABEL_COLUMNS].value_counts())

7103

In [7]:
df_train["len_text"] = df_train["text"].apply(lambda text: len(text.split()))
print(df_train.len_text.max(), df_train.len_text.mean())
df_val["len_text"] = df_val["text"].apply(lambda text: len(text.split()))
print(df_val.len_text.max(), df_val.len_text.mean())
df_test["len_text"] = df_test["text"].apply(lambda text: len(text.split()))
print(df_test.len_text.max(), df_test.len_text.mean())

765 28.552724201041812
583 29.1316685584563
702 27.480681818181818


In [8]:
X_train, y_train, X_val, y_val, X_test, y_test = df_train["text"], df_train[LABEL_COLUMNS], df_val["text"], df_val[LABEL_COLUMNS], df_test["text"], df_test[LABEL_COLUMNS]

In [9]:
import lightning as L
from transformers import DistilBertModel, DistilBertTokenizer, AdamW
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
from torchmetrics.classification import MultilabelF1Score
from torchmetrics.classification import MultilabelAUROC
import torch.nn as nn

import numpy as np
from functools import partial
import os

In [10]:
def collate_fn(batch):
    # print("I am in collate_fn")
    input_ids = torch.stack([torch.LongTensor(sample["input_ids"]) for sample in batch])
    attention_mask = torch.stack([torch.Tensor(sample["attention_mask"]) for sample in batch])
    labels = [sample["labels"] for sample in batch]
    labels_tensor = [torch.tensor([0.0] * n_labels) for _ in labels]
    padded_tensors = []
    for label, label_tensor in zip(labels, labels_tensor):
        for l in label:
            label_tensor[int(l)] = 1.0

        padded_tensors.append(label_tensor.to(torch.float32))
    stacked_labels = torch.stack(padded_tensors)

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': stacked_labels}

In [11]:
# step 1 - class of basic data structure

class MultiLabelBasicDataset(Dataset):
    def __init__(self, encodings):
        print("I am in MultiLabelDataset.__init__")
        self.encodings = encodings
        self.dataset_tokenized = Dataset.from_dict(self.encodings)
        self.labels = self.encodings["labels"]

    # def __getitem__(self, idx):
    #     print("I am in __getitem__")
    #     item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    #     item['labels'] = self.labels[idx]
    #     return item

    # def __len__(self):
    #     return len(self.labels)


In [12]:
# Step 2: All datasets (train, validation, and test)

class MultiLabelDataModule(L.LightningDataModule):
    def __init__(self, tokenizer, texts_train, labels_train, texts_val, labels_val, texts_test, labels_test, batch_size=batch_size, max_token_len=max_length):
        super().__init__()
        print("I am in MultiLabelDataModule.__init__()")
        self.tokenizer = tokenizer
        self.texts_train = texts_train
        self.labels_train = labels_train
        self.texts_val = texts_val
        self.labels_val = labels_val
        self.texts_test = texts_test
        self.labels_test = labels_test
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def tokenize(self, texts, labels):
        print("tokenize func")
        input_ids, attention_mask = [], []
        text_tokenized=self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        input_ids += text_tokenized["input_ids"]
        attention_mask += text_tokenized["attention_mask"]
        labels_np = labels.to_numpy().astype(np.float32)
        labels_indices = [np.where(row == 1)[0].astype(np.float32).tolist() for row in labels_np]

        ret = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels_indices,
        }

        print("tokenize finished")
        return ret

    def setup(self, stage=None):
        # Split the dataset here if needed, for now assuming texts and labels are ready
        # Tokenize the text
        print("I am in setup")

        # Create dataset train
        print("Create dataset train")
        self.encodings_train = self.tokenize(self.texts_train, self.labels_train)
        self.dataset_train = MultiLabelBasicDataset(self.encodings_train).dataset_tokenized

        # Create dataset val
        print("Create dataset val")
        self.encodings_val = self.tokenize(self.texts_val, self.labels_val)
        self.dataset_val = MultiLabelBasicDataset(self.encodings_val).dataset_tokenized

        # Create dataset test
        print("Create dataset test")
        self.encodings_test = self.tokenize(self.texts_test, self.labels_test)
        self.dataset_test = MultiLabelBasicDataset(self.encodings_test).dataset_tokenized

    def train_dataloader(self):
        # Implement logic to split dataset if needed or use full dataset for training
        print("I am in train_dataloader")
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True, collate_fn=collate_fn, num_workers=num_workers)

    def val_dataloader(self):
        # Implement logic to split dataset if needed or use a portion for validation
        # Placeholder: using the same dataset for simplicity, replace with actual validation set
        print("I am in val_dataloader")
        return DataLoader(self.dataset_val, batch_size=self.batch_size, collate_fn=collate_fn, num_workers=num_workers)

    def test_dataloader(self):
        # Implement logic to split dataset if needed or use a portion for validation
        # Placeholder: using the same dataset for simplicity, replace with actual validation set
        print("I am in test_dataloader")
        return DataLoader(self.dataset_test, batch_size=self.batch_size, collate_fn=collate_fn, num_workers=num_workers)


In [13]:
# Step 3 - build the model

class MultiLabelDetectionModel(L.LightningModule):
    def __init__(self, model_name=model_name, n_labels = n_labels):
        print("I am in MultiLabelDetectionModel.__init__")
        super().__init__()
        self.save_hyperparameters()
        self.n_labels = n_labels
        print("I am after: save_hyperparameters")
        self.tokenizer = DistilBertTokenizer.from_pretrained(model_name)
        # self.tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
        print("I am after: tokenizer init")
        self.model = DistilBertModel.from_pretrained(model_name)
        # self.model = DistilBertForTokenClassification.from_pretrained(model_name)
        # self.model = DistilBertForSequenceClassification.from_pretrained(model_name)
        print("I am after: model init")
        print("self.model.config.hidden_size:")
        print(self.model.config.hidden_size)
        self.classifier = nn.Linear(self.model.config.hidden_size, n_labels)
        print("I am after: nn.Linear(self.model.config.hidden_size, n_labels)")
        self.criterion = nn.BCEWithLogitsLoss()
        print("I am after: nn.BCEWithLogitsLoss()")
        self.auroc = MultilabelAUROC(n_labels)
        print("I am after: MultilabelAUROC()")
        self.multi_label_f1 = MultilabelF1Score(n_labels)
        print("I am after: multi_label_f1")

        self.train_loss = []
        self.train_predictions = []
        self.train_labels = []
        self.val_loss = []
        self.val_predictions = []
        self.val_labels = []
        self.test_loss = []
        self.test_predictions = []
        self.test_labels = []

    def forward(self, input_ids, attention_mask):
        # print("I am in forward")
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        pooler = last_hidden_state[:, 0]
        logits = self.classifier(pooler)

        return logits

    def training_step(self, batch, batch_idx):
        # print("I am in training_step")
        input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["labels"]
        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.sigmoid(logits)  # Apply sigmoid to convert logits to probabilities
        loss = self.criterion(predictions, labels)
        self.train_loss.append(loss)
        # if self.train_loss:
        self.log("training/loss_epoch:", torch.stack(self.train_loss).mean(), on_step=False, on_epoch=True, logger=True)
        self.train_predictions.extend(predictions)
        self.train_labels.extend(labels)

        return loss

    def on_training_epoch_end(self, outputs):
        print("I am in on_training_epoch_end")
        print("outputs: ")
        print(outputs)
        if self.train_loss:
            avg_loss = torch.stack(self.train_loss).mean()
            self.log("training/loss_epoch:", avg_loss, prog_bar=True, logger=True)

        self.train_predictions = torch.stack(self.train_predictions)
        self.train_labels = torch.stack(self.train_labels).long()
        multi_label_f1_score = self.multi_label_f1(self.train_predictions, self.train_labels)
        print("training/multi label f1: ", multi_label_f1_score)
        self.log("training/multi label f1: ", multi_label_f1_score, prog_bar=True, logger=True)
        multi_label_auroc_score = self.auroc(self.train_predictions, self.train_labels)
        print("training/multi label auroc: ", multi_label_auroc_score)
        self.log("training/multi label auroc: ", multi_label_auroc_score, prog_bar=True, logger=True)

        self.train_loss = []
        self.train_predictions = []
        self.train_labels = []

    def validation_step(self, batch, batch_idx):
        # print("I am in validation_step")
        input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["labels"]
        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.sigmoid(logits)  # Apply sigmoid to convert logits to probabilities
        loss = self.criterion(predictions, labels)
        self.val_loss.append(loss)
        avg_loss = torch.stack(self.val_loss).mean()
        self.log("validation/loss_epoch:", avg_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        self.val_predictions.extend(predictions)
        self.val_labels.extend(labels)

        return loss

    def on_validation_epoch_end(self):
        print("I am in on_validation_epoch_end")
        if self.val_loss:
            avg_loss = torch.stack(self.val_loss).mean()
            print("validation/loss_epoch:", avg_loss)
            self.log("validation/loss_epoch:", avg_loss, prog_bar=True, logger=True)

        self.val_predictions = torch.stack(self.val_predictions)
        self.val_labels = torch.stack(self.val_labels).long()
        multi_label_f1_score = self.multi_label_f1(self.val_predictions, self.val_labels)
        print("validation/multi label f1: ", multi_label_f1_score)
        self.log("validation/multi label f1: ", multi_label_f1_score, prog_bar=True, logger=True)
        multi_label_auroc_score = self.auroc(self.val_predictions, self.val_labels)
        print("validation/multi label auroc: ", multi_label_auroc_score)
        self.log("validation/multi label auroc: ", multi_label_auroc_score, prog_bar=True, logger=True)

        self.val_loss = []
        self.val_predictions = []
        self.val_labels = []

    def test_step(self, batch, batch_idx):
        # print("I am in test_step")
        input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["labels"]
        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.sigmoid(logits)  # Apply sigmoid to convert logits to probabilities
        loss = self.criterion(predictions, labels)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.test_predictions.extend(predictions)
        self.test_labels.extend(labels)

        return loss

    def on_test_epoch_end(self):
        print("I am in on_test_epoch_end")
        if self.test_loss:
            avg_loss = torch.stack(self.test_loss).mean()
            print("test/loss_epoch:", avg_loss)
            self.log("test/loss_epoch:", avg_loss, prog_bar=True, logger=True)

        self.test_predictions = torch.stack(self.test_predictions)
        self.test_labels = torch.stack(self.test_labels).long()
        multi_label_f1_score = self.multi_label_f1(self.test_predictions, self.test_labels)
        print("test/multi label f1: ", multi_label_f1_score)
        self.log("test/multi label f1: ", multi_label_f1_score, prog_bar=True, logger=True)
        multi_label_auroc_score = self.auroc(self.test_predictions, self.test_labels)
        print("test/multi label auroc: ", multi_label_auroc_score)
        self.log("test/multi label auroc: ", multi_label_auroc_score, prog_bar=True, logger=True)

        self.test_loss = []
        self.test_predictions = []
        self.test_labels = []

    def predict(self, text):
        print("I am in predict")
        inputs = self.tokenizer(text, return_tensors='pt')

        rets = []
        with torch.no_grad():
            outputs = self.model(**inputs)
            last_hidden_state = outputs[0]
            pooler = last_hidden_state[:, 0]
            logits = self.classifier(pooler)
            predictions = torch.sigmoid(logits)
            print("predictions: ")
            print(predictions)

            for i, prediction in enumerate(np.array(predictions[0])):
                if prediction >= 0.5:
                    rets.append(LABEL_COLUMNS[i].replace("label_", "").replace("_", " "))

            if len(rets) == 0:
                ret = "This message was approved"
            elif 1 == len(rets):
                ret = f"This message was {rets[0]}"
            elif 2 == len(rets):
                ret = f"This message was {rets[0]} and {rets[1]}"
            else:
                ret = f"This message was {rets[0]}, {rets[1]}, and {rets[2]}"

        return ret

    def configure_optimizers(self):
        print(f"I am in configure_optimizers - learning_rate: {learning_rate}")  # self.hparams.lr: {self.hparams.lr} -
        optimizer = torch.optim.Adam(params = self.parameters(), lr=learning_rate)

        return optimizer

In [14]:
# Download the model
#         Create instances of your model and data module, then initiate a trainer to start training:
#         python
#

# ***run only once***
model_wrapper = MultiLabelDetectionModel()

I am in MultiLabelDetectionModel.__init__
I am after: save_hyperparameters


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

I am after: tokenizer init


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

I am after: model init
self.model.config.hidden_size:
768
I am after: nn.Linear(self.model.config.hidden_size, n_labels)
I am after: nn.BCEWithLogitsLoss()
I am after: MultilabelAUROC()
I am after: multi_label_f1


In [15]:
data_module = MultiLabelDataModule(tokenizer=model_wrapper.tokenizer, texts_train=list(X_train), labels_train=y_train, texts_val=list(X_val), labels_val=y_val, texts_test=list(X_test), labels_test=y_test)

I am in MultiLabelDataModule.__init__()


In [16]:
wandb_logger = WandbLogger(project=wandb_project, name=wandb_name, group="yoav_yosef")

In [17]:
trainer = L.Trainer(max_epochs=epochs, devices=1 if torch.cuda.is_available() else 0, accelerator="auto", logger=wandb_logger)
# trainer = L.Trainer(max_epochs=epcohs, logger=wandb_logger)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [18]:
trainer.fit(model_wrapper, data_module)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.


I am in setup
Create dataset train
tokenize func
tokenize finished
I am in MultiLabelDataset.__init__
Create dataset val
tokenize func
tokenize finished
I am in MultiLabelDataset.__init__
Create dataset test
tokenize func


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


tokenize finished
I am in MultiLabelDataset.__init__


INFO: 
  | Name           | Type              | Params
-----------------------------------------------------
0 | model          | DistilBertModel   | 66.4 M
1 | classifier     | Linear            | 2.3 K 
2 | criterion      | BCEWithLogitsLoss | 0     
3 | auroc          | MultilabelAUROC   | 0     
4 | multi_label_f1 | MultilabelF1Score | 0     
-----------------------------------------------------
66.4 M    Trainable params
0         Non-trainable params
66.4 M    Total params
265.461   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name           | Type              | Params
-----------------------------------------------------
0 | model          | DistilBertModel   | 66.4 M
1 | classifier     | Linear            | 2.3 K 
2 | criterion      | BCEWithLogitsLoss | 0     
3 | auroc          | MultilabelAUROC   | 0     
4 | multi_label_f1 | MultilabelF1Score | 0     
-----------------------------------------------------
66.4 M    Trainable par

...

Validation: |          | 0/? [00:00<?, ?it/s]

I am in on_validation_epoch_end
validation/loss_epoch: tensor(0.6303, device='cuda:0')
validation/multi label f1:  tensor(0.7724, device='cuda:0')
validation/multi label auroc:  tensor(0.9306, device='cuda:0')


Validation: |          | 0/? [00:00<?, ?it/s]

I am in on_validation_epoch_end
validation/loss_epoch: tensor(0.6327, device='cuda:0')
validation/multi label f1:  tensor(0.8065, device='cuda:0')
validation/multi label auroc:  tensor(0.9273, device='cuda:0')


INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [19]:
test_loader = DataLoader(data_module.dataset_test, batch_size=batch_size, collate_fn=collate_fn)
trainer.test(model_wrapper, test_loader)

I am in setup
Create dataset train
tokenize func
tokenize finished
I am in MultiLabelDataset.__init__
Create dataset val
tokenize func
tokenize finished
I am in MultiLabelDataset.__init__
Create dataset test
tokenize func


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


tokenize finished
I am in MultiLabelDataset.__init__


Testing: |          | 0/? [00:00<?, ?it/s]

I am in on_test_epoch_end
test/multi label f1:  tensor(0.7930, device='cuda:0')
test/multi label auroc:  tensor(0.9304, device='cuda:0')


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  test/multi label auroc:  │    0.9304090738296509     │
│   test/multi label f1:    │    0.7929986119270325     │
│         test_loss         │    0.6311406493186951     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6311406493186951,
  'test/multi label f1: ': 0.7929986119270325,
  'test/multi label auroc: ': 0.9304090738296509}]

In [20]:
model_wrapper.predict("Hi!")

I am in predict
predictions: 
tensor([[0.0010, 0.0012, 0.0007]])


'This message was approved'

In [21]:
model_wrapper.predict("I hate every one!")

I am in predict
predictions: 
tensor([[0.0035, 0.0009, 0.5447]])


'This message was bullying'

In [22]:
model_wrapper.predict("The Jews control the world, they should be killed!")

I am in predict
predictions: 
tensor([[0.0026, 0.9981, 0.9190]])


'This message was hate speech and bullying'

In [23]:
model_wrapper.predict("Shut up nigger!")

I am in predict
predictions: 
tensor([[0.9972, 0.9999, 0.9966]])


'This message was nsfw, hate speech, and bullying'

In [24]:
model_wrapper.predict("I will kick the shit out of you, retarded")  # un balanced data set

I am in predict
predictions: 
tensor([[0.9924, 0.0027, 0.9995]])


'This message was nsfw and bullying'

In [25]:
model_wrapper.predict("Do you want a bigger penis?")  # un balanced data set

I am in predict
predictions: 
tensor([[0.9989, 0.0053, 0.0017]])


'This message was nsfw'

In [26]:
model_wrapper.predict("Shut up you piece of shit!")

I am in predict
predictions: 
tensor([[0.9982, 0.0011, 0.9984]])


'This message was nsfw and bullying'

In [27]:
model_wrapper.predict("Watch free porn now!")

I am in predict
predictions: 
tensor([[9.5197e-01, 1.0131e-02, 1.2825e-04]])


'This message was nsfw'

In [28]:
model_wrapper.predict("You are a nigger!")

I am in predict
predictions: 
tensor([[0.9972, 0.9999, 0.9872]])


'This message was nsfw, hate speech, and bullying'

In [30]:
# model_wrapper.save_model(f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/model_wrapper.ckpt")
torch.save(model_wrapper, f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/model_wrapper.ckpt")

In [31]:
import pickle
from google.colab import files

pickle.dump(model_wrapper.model, open(f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/model_{pickle_name}.pkl", "wb"))
pickle.dump(model_wrapper.tokenizer, open(f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/tokenizer_{pickle_name}.pkl", "wb"))


In [32]:
import torch

# Save the model
torch.save(model_wrapper.model.state_dict(), f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/model_{pickle_name}.pth")

In [33]:
model_wrapper.tokenizer.save_pretrained(f"/content/drive/MyDrive/BIU - Final Project/Models/{task_folder}/tokenizer_{pickle_name}.pth")

('/content/drive/MyDrive/BIU - Final Project/Models/Multi Label Hate Speech Bullying NSFW/tokenizer_kaggle - distilbert-base-uncased_only_one_top_layer_train_sample_balanced toxic data set_epochs_10_batch_size_16_max_length_500_learning_rate_1e-05_num_workers_15_with_all_accuracy.pth/tokenizer_config.json',
 '/content/drive/MyDrive/BIU - Final Project/Models/Multi Label Hate Speech Bullying NSFW/tokenizer_kaggle - distilbert-base-uncased_only_one_top_layer_train_sample_balanced toxic data set_epochs_10_batch_size_16_max_length_500_learning_rate_1e-05_num_workers_15_with_all_accuracy.pth/special_tokens_map.json',
 '/content/drive/MyDrive/BIU - Final Project/Models/Multi Label Hate Speech Bullying NSFW/tokenizer_kaggle - distilbert-base-uncased_only_one_top_layer_train_sample_balanced toxic data set_epochs_10_batch_size_16_max_length_500_learning_rate_1e-05_num_workers_15_with_all_accuracy.pth/vocab.txt',
 '/content/drive/MyDrive/BIU - Final Project/Models/Multi Label Hate Speech Bullyin